In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

from matplotlib import rc
rc("font", family = "Malgun Gothic")

get_ipython().run_line_magic("matplotlib", "inline")

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

import time
from tqdm.notebook import tqdm

from glob import glob
import warnings
warnings.filterwarnings('ignore')

# 1. 하나로 합치기

### 1-1) 위도, 경도 값 추가

In [6]:
a_3 = pd.read_csv('./tmp_stop.csv', index_col=0)
a_4 = pd.read_csv('./tmp_stop_2.csv', index_col=0)
a_5 = pd.read_csv('./tmp_stop_3.csv', index_col=0)
a_6 = pd.read_csv('./tmp_stop_4.csv', index_col=0)
a_7 = pd.read_csv('./tmp_stop_5.csv', index_col=0)
a_8 = pd.read_csv('./tmp_stop_6.csv', index_col=0)
a_9 = pd.read_csv('./tmp_stop_7.csv', index_col=0)
a_10 = pd.read_csv('./tmp_stop_8.csv', index_col=0)

a_12 = pd.read_csv('./tmp_stop_10.csv', index_col=0)
a_13 = pd.read_csv('./tmp_stop_11.csv', index_col=0)
a_14 = pd.read_csv('./tmp_stop_12.csv', index_col=0)
a_15 = pd.read_csv('./tmp_stop_13.csv', index_col=0)
a_16 = pd.read_csv('./tmp_stop_14.csv', index_col=0)

In [11]:
aa = pd.concat([a_3, a_4, a_5, a_6, a_7, a_8, a_9, a_10, a_12, a_13, a_14, a_15, a_16])
aa.head()

,1,2,3,4,name
128884,경기도,수원시,장안구,정자동,로얄팰리스
128885,경기도,수원시,장안구,이목동,이목동.장안힐스테이트
128886,경기도,수원시,장안구,이목동,이목동.장안STX칸
128887,경기도,수원시,장안구,송죽동,연세수요양병원.송죽희성연인아파트
128888,경기도,수원시,장안구,송죽동,만석공원테니스장


In [14]:
aa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108637 entries, 128884 to 4999
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   1       108637 non-null  object
 1   2       108074 non-null  object
 2   3       108637 non-null  object
 3   4       108637 non-null  object
 4   name    108637 non-null  object
dtypes: object(5)
memory usage: 5.0+ MB


In [16]:
df = pd.read_excel('./bus_station_3.xlsx', index_col=0)
df.drop(columns=["광역시/도", "시/군", "구","동/읍/면"], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133459 entries, 116612 to 250070
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Stop_NM  133459 non-null  object 
 1   Lati     133459 non-null  float64
 2   Long     133459 non-null  float64
dtypes: float64(2), object(1)
memory usage: 4.1+ MB


In [25]:
bus = pd.merge(aa, df, left_on='name', right_on='Stop_NM')
bus.rename(columns={"1":"광역시/도", "2":"시/군", "3":"구","4":"동/읍/면"}, inplace=True)
bus.drop(columns=['name'], axis=1, inplace=True)
bus

,광역시/도,시/군,구,동/읍/면,Stop_NM,Lati,Long
0,경기도,수원시,장안구,정자동,로얄팰리스,37.295667,127.007567
1,경기도,수원시,장안구,정자동,로얄팰리스,37.295400,127.008283
2,경기도,수원시,장안구,정자동,로얄팰리스,37.295667,127.007567
3,경기도,수원시,장안구,정자동,로얄팰리스,37.295400,127.008283
4,경기도,수원시 장안구,수원시 장안구,송죽동,로얄팰리스,37.295667,127.007567
...,...,...,...,...,...,...,...
511911,경기도,양평군,양평군,옥천면,용천3리.발해,37.539733,127.493850
511912,경기도,양평군,양평군,옥천면,용천3리종점,37.553933,127.502350
511913,경기도,양평군,양평군,옥천면,용천3리종점,37.553933,127.502350
511914,경기도,양평군,양평군,양동면,양동역,37.420417,127.754050


In [26]:
bus_df = bus[~bus.duplicated(subset=['Lati', 'Long'])]
bus_df

,광역시/도,시/군,구,동/읍/면,Stop_NM,Lati,Long
0,경기도,수원시,장안구,정자동,로얄팰리스,37.295667,127.007567
1,경기도,수원시,장안구,정자동,로얄팰리스,37.295400,127.008283
2,경기도,수원시,장안구,정자동,로얄팰리스,37.295667,127.007567
3,경기도,수원시,장안구,정자동,로얄팰리스,37.295400,127.008283
12,경기도,수원시,장안구,이목동,이목동.장안힐스테이트,37.313450,126.985417
...,...,...,...,...,...,...,...
511907,경기도,양평군,양평군,옥천면,용천3리.발해,37.539733,127.493850
511912,경기도,양평군,양평군,옥천면,용천3리종점,37.553933,127.502350
511913,경기도,양평군,양평군,옥천면,용천3리종점,37.553933,127.502350
511914,경기도,양평군,양평군,양동면,양동역,37.420417,127.754050


In [27]:
bus_df.to_csv('./버스정류장위치.csv', encoding='utf-8')

----

In [35]:
df2 = pd.read_excel('./bus_station_2.xlsx', index_col=0)
b = pd.read_csv('./final_data/bus_stop.csv', index_col=0)

In [42]:
df2.drop(columns=['광역시/도', '시/군', '구', '동/읍/면'], axis=1, inplace=True)
df2.head()

,Stop_NM,Lati,Long
5615,성원아파트,37.55653,126.90162
5616,동교초등학교,37.55681,126.90295
5617,하나은행망원동지점,37.55709,126.90428
5618,망원시장,37.55744,126.90598
5619,화평교회,37.55444,126.90116


In [39]:
b.drop(columns=['Lati', "Long"], inplace=True)
b.head()

,광역시/도,시/군,구,동/읍/면,Stop_NM
0,서울특별시,마포구,마포구,망원동,성원아파트
1,서울특별시,마포구,마포구,망원동,동교초등학교
2,서울특별시,마포구,마포구,망원동,하나은행망원동지점
3,서울특별시,마포구,마포구,망원동,망원시장
4,서울특별시,마포구,마포구,망원동,화평교회


In [44]:
bus2 = pd.merge(b, df2, left_on='Stop_NM', right_on='Stop_NM')
bus_df2 = bus2[~bus2.duplicated(subset=['Lati', 'Long'])]

,광역시/도,시/군,구,동/읍/면,Stop_NM,Lati,Long
0,서울특별시,마포구,마포구,망원동,성원아파트,37.55653,126.90162
1,서울특별시,마포구,마포구,망원동,성원아파트,37.55671,126.90157
2,서울특별시,마포구,마포구,망원동,성원아파트,37.50251,126.89667
3,서울특별시,마포구,마포구,망원동,성원아파트,37.50882,126.92542
4,서울특별시,마포구,마포구,망원동,성원아파트,37.77233,126.74738
...,...,...,...,...,...,...,...
195011,경기도,여주시,여주시,금사면,소유리,37.41738,127.49493
195024,경기도,여주시,여주시,북내면,고달사지,37.39213,127.66177
195025,경기도,여주시,여주시,북내면,고달사지,37.39197,127.66202
195028,경기도,여주시,여주시,여흥동,프레미엄아울렛,37.24118,127.61343


In [45]:
bus_df2 = bus_df2[['Stop_NM', 'Lati', 'Long', '광역시/도', '시/군', '구', '동/읍/면']]

### 1-2) 모두 통합

In [30]:
bus_df = bus_df[['Stop_NM', 'Lati', 'Long', '광역시/도', '시/군', '구', '동/읍/면']]

In [8]:
a_1 = pd.read_excel('./station_6_완.xlsx', index_col=0)
a_2 = pd.read_excel('./bus_station_1.xlsx', index_col=0)

In [46]:
bus_stop = pd.concat([a_1, a_2, bus_df, bus_df2])
bus_stop

,Stop_NM,Lati,Long,광역시/도,시/군,구,동/읍/면
116612,이천,35.266479,129.238168,부산광역시,0,기장군,일광읍
116613,이동방파제,35.273316,129.246587,부산광역시,0,기장군,일광읍
116614,이천,35.266642,129.238113,부산광역시,0,기장군,일광읍
116615,동해선일광역,35.265694,129.233328,부산광역시,0,기장군,일광읍
116616,이동,35.272060,129.243942,부산광역시,0,기장군,일광읍
...,...,...,...,...,...,...,...
195011,소유리,37.417380,127.494930,경기도,여주시,여주시,금사면
195024,고달사지,37.392130,127.661770,경기도,여주시,여주시,북내면
195025,고달사지,37.391970,127.662020,경기도,여주시,여주시,북내면
195028,프레미엄아울렛,37.241180,127.613430,경기도,여주시,여주시,여흥동


In [47]:
bus_stop.to_csv('./final_data/버정주소통합본.csv', encoding='utf-8')

# 2. 서울, 경기, 인천 데이터만 추출

In [65]:
bus_stop = pd.read_csv('./final_data/버정주소통합본.csv', index_col=0)

In [66]:
bus_stop = bus_stop[bus_stop['광역시/도'].isin(['서울특별시', '인천광역시', '경기도'])].reset_index(drop=True)
bus_stop

,Stop_NM,Lati,Long,광역시/도,시/군,구,동/읍/면
0,종로2가사거리(중),37.569783,126.987733,서울특별시,0,종로구,종로2가
1,종로2가.삼일교(중),37.568683,126.987533,서울특별시,0,종로구,관철동
2,광화문,37.570033,126.976250,서울특별시,0,종로구,세종로
3,서울지방경찰청.경복궁역,37.575817,126.971483,서울특별시,0,종로구,내자동
4,광화문.광화문빌딩,37.569517,126.976750,서울특별시,0,종로구,세종로
...,...,...,...,...,...,...,...
157900,소유리,37.417380,127.494930,경기도,여주시,여주시,금사면
157901,고달사지,37.392130,127.661770,경기도,여주시,여주시,북내면
157902,고달사지,37.391970,127.662020,경기도,여주시,여주시,북내면
157903,프레미엄아울렛,37.241180,127.613430,경기도,여주시,여주시,여흥동


# 3. 시군구 컬럼 정리

### 3-1) '시군구' 컬럼으로 통합
- 행정구가 있는 ("수원시", "성남시", '안양시', '안산시', '고양시', '용인시') 만 OO시 OO구 형식으로 통합

In [73]:
for idx, row in bus_stop.iterrows():
    if row['시/군'] in ("수원시", "성남시", '안양시', '안산시', '고양시', '용인시'):
        if len(row['구'].split()) == 1:
            bus_stop.iloc[idx, 5] = row['시/군'] + " " + row['구']

### 3-2) 시/군, 구 가 모두 빈칸인 데이터 채워주기

In [77]:
e_list = []
for idx, row in bus_stop.iterrows():
      if (row['시/군'] == "0") and row['구'] == "0" :
            e_list.append(idx)

In [79]:
bus_stop.loc[e_list, :]

,Stop_NM,Lati,Long,광역시/도,시/군,구,동/읍/면
128115,종근당,37.559560,126.964480,서울특별시,0,0,NaN
128116,종근당,37.469040,126.697820,서울특별시,0,0,NaN
128117,종근당,37.469410,126.697900,서울특별시,0,0,NaN
128118,종근당,37.559827,126.962918,서울특별시,0,0,NaN
128119,동교동삼거리,37.557810,126.929530,서울특별시,0,0,NaN
...,...,...,...,...,...,...,...
133425,드무리해변,37.252860,126.516330,인천광역시,0,0,NaN
133426,드무리해변,37.252700,126.516390,인천광역시,0,0,NaN
133427,큰산너머입구,37.244800,126.527210,인천광역시,0,0,NaN
133428,큰산너머입구,37.244770,126.527140,인천광역시,0,0,NaN


In [80]:
bus_stop.to_csv('./final_data/버정주소통합본.csv', encoding='utf-8')

### 3-3) 행정구명 정리

In [2]:
bus_stop = pd.read_excel('./final_data/버정최종.xlsx', index_col=0)
bus_stop = bus_stop[bus_stop["광역시/도"].isin(['서울특별시', "인천광역시", "경기도"])]
bus_stop

,Stop_NM,Lati,Long,광역시/도,시/군,구,동/읍/면,동
0,종로2가사거리(중),37.569783,126.987733,서울특별시,0,종로구,종로2가,NaN
1,종로2가.삼일교(중),37.568683,126.987533,서울특별시,0,종로구,관철동,NaN
2,광화문,37.570033,126.976250,서울특별시,0,종로구,세종로,NaN
3,서울지방경찰청.경복궁역,37.575817,126.971483,서울특별시,0,종로구,내자동,NaN
4,광화문.광화문빌딩,37.569517,126.976750,서울특별시,0,종로구,세종로,NaN
...,...,...,...,...,...,...,...,...
157900,소유리,37.417380,127.494930,경기도,여주시,여주시,금사면,NaN
157901,고달사지,37.392130,127.661770,경기도,여주시,여주시,북내면,NaN
157902,고달사지,37.391970,127.662020,경기도,여주시,여주시,북내면,NaN
157903,프레미엄아울렛,37.241180,127.613430,경기도,여주시,여주시,여흥동,NaN


In [3]:
bus_stop.fillna("0")
bus_stop["시/군"] = bus_stop["시/군"].astype(str)

In [4]:
bus_stop.reset_index(drop=True, inplace=True)
bus_stop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157499 entries, 0 to 157498
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Stop_NM  157499 non-null  object 
 1   Lati     157499 non-null  float64
 2   Long     157499 non-null  float64
 3   광역시/도    157499 non-null  object 
 4   시/군      157499 non-null  object 
 5   구        157499 non-null  object 
 6   동/읍/면    151642 non-null  object 
 7   동        4898 non-null    object 
dtypes: float64(2), object(6)
memory usage: 9.6+ MB


In [5]:
e_list=[]
for idx, row in tqdm(bus_stop.iterrows()):
    if (row["광역시/도"] == "경기도") and (len(row["시/군"].split()) == 2):
        try:
            bus_stop.loc[idx, "시/군"] = row["시/군"].split()[0]
        except:
            e_list.append(idx)

0it [00:00, ?it/s]

### 3-4) 0값 정리

In [18]:
for idx, row in tqdm(bus_stop.iterrows()):
    if row["시/군"] == "0":
        bus_stop.loc[idx, "시/군"] = row['구']

0it [00:00, ?it/s]

In [19]:
bus_stop.to_excel('./bus_stop_raw.xlsx', encoding='utf-8')

---

In [2]:
Stop_bus_2019 = pd.read_excel("./2019.03/201903_1_버스_1_BaseInfo.xlsx", sheet_name=0)
Route_bus_2019 = pd.read_excel("./2019.03/201903_1_버스_1_BaseInfo.xlsx", sheet_name=1)
RouteStop_bus_2019 = pd.read_excel("./2019.03/201903_1_버스_1_BaseInfo.xlsx", sheet_name=2)
StopTime_bus_2019 = pd.read_csv("./2019.03/201903_1_버스_2_StopTime.txt")

Stop_bus_2020 = pd.read_excel("./2020.03/202003_1_버스_1_BaseInfo.xlsx", sheet_name=0)
Route_bus_2020 = pd.read_excel("./2020.03/202003_1_버스_1_BaseInfo.xlsx", sheet_name=1)
RouteStop_bus_2020 = pd.read_excel("./2020.03/202003_1_버스_1_BaseInfo.xlsx", sheet_name=2)
StopTime_bus_2020 = pd.read_csv("./2020.03/202003_1_버스_2_StopTime.txt")

Stop_bus_2021 = pd.read_excel("./2021.03/202103_1_버스_1_BaseInfo.xlsx", sheet_name=0)
Route_bus_2021 = pd.read_excel("./2021.03/202103_1_버스_1_BaseInfo.xlsx", sheet_name=1)
RouteStop_bus_2021 = pd.read_excel("./2021.03/202103_1_버스_1_BaseInfo.xlsx", sheet_name=2)
StopTime_bus_2021 = pd.read_csv("./2021.03/202103_1_버스_2_StopTime.txt")

# 4. 지역별 지하철역 구분하기
- 통행량 데이터의 지역구분에 따른다.
- 크롤링으로 구한 지하철역의 주소 데이터 사용

In [11]:
order_list = ['경기도 가평군', '경기도 고양시 ', '경기도 과천시', '경기도 광명시', '경기도 광주시', '경기도 구리시',
       '경기도 군포시', '경기도 김포시', '경기도 남양주시', '경기도 동두천시', '경기도 부천시', '경기도 성남시 ',
       '경기도 수원시 ', '경기도 시흥시', '경기도 안산시 ', '경기도 안성시', '경기도 안양시 ', '경기도 양주시',
       '경기도 양평군', '경기도 여주시', '경기도 연천군', '경기도 오산시', '경기도 용인시 ', '경기도 의왕시',
       '경기도 의정부시', '경기도 이천시', '경기도 파주시', '경기도 평택시', '경기도 포천시', '경기도 하남시',
       '경기도 화성시', '서울특별시 강남구', '서울특별시 강동구', '서울특별시 강북구', '서울특별시 강서구',
       '서울특별시 관악구', '서울특별시 광진구', '서울특별시 구로구', '서울특별시 금천구', '서울특별시 노원구',
       '서울특별시 도봉구', '서울특별시 동대문구', '서울특별시 동작구', '서울특별시 마포구', '서울특별시 서대문구',
       '서울특별시 서초구', '서울특별시 성동구', '서울특별시 성북구', '서울특별시 송파구', '서울특별시 양천구',
       '서울특별시 영등포구', '서울특별시 용산구', '서울특별시 은평구', '서울특별시 종로구', '서울특별시 중구',
       '서울특별시 중랑구', '인천광역시 강화군', '인천광역시 계양구', '인천광역시 남동구', '인천광역시 동구',
       '인천광역시 미추홀구', '인천광역시 부평구', '인천광역시 서구', '인천광역시 연수구', '인천광역시 옹진군',
       '인천광역시 중구']

In [14]:
bus_stop = pd.read_excel('./bus_stop_raw.xlsx', index_col=0)
bus_stop.drop(columns=['동'], axis=1, inplace=True)
bus_stop['지역'] = bus_stop['광역시/도'] + " " + bus_stop['시/군']
bus_stop

,Stop_NM,Lati,Long,광역시/도,시/군,구,동/읍/면,지역
0,종로2가사거리(중),37.569783,126.987733,서울특별시,종로구,종로구,종로2가,서울특별시 종로구
1,종로2가.삼일교(중),37.568683,126.987533,서울특별시,종로구,종로구,관철동,서울특별시 종로구
2,광화문,37.570033,126.976250,서울특별시,종로구,종로구,세종로,서울특별시 종로구
3,서울지방경찰청.경복궁역,37.575817,126.971483,서울특별시,종로구,종로구,내자동,서울특별시 종로구
4,광화문.광화문빌딩,37.569517,126.976750,서울특별시,종로구,종로구,세종로,서울특별시 종로구
...,...,...,...,...,...,...,...,...
157494,소유리,37.417380,127.494930,경기도,여주시,여주시,금사면,경기도 여주시
157495,고달사지,37.392130,127.661770,경기도,여주시,여주시,북내면,경기도 여주시
157496,고달사지,37.391970,127.662020,경기도,여주시,여주시,북내면,경기도 여주시
157497,프레미엄아울렛,37.241180,127.613430,경기도,여주시,여주시,여흥동,경기도 여주시


---
# 5. 정류장별 버스 노선
- 각 정류장에 오는 버스들을 알아내기
- Stop 파일에 있는 정류장들을 기준으로, 각 정류장에 오는 버스를 알아야 한다.
- RouteStop 파일과 merge하여 각 정류장에 서는 버스를 알아낸다.
- RouteStop 파일에서 Stop_YN = TRUE 인 데이터만 사용한다.(경유하는건 사용하지 않는다)
- 버스번호는 "RT_NM" 인데, Stop과 RouteStop에는 없어서,,, 다시 Route 파일과 merge한다
- 서울경기인천 내 정류장들만 추출

In [34]:
s_list = list(bus_stop['Stop_NM'].unique())

In [50]:
line_2021 = pd.merge(Stop_bus_2021, RouteStop_bus_2021[RouteStop_bus_2021["Stop_YN"] == "Y"], on="Stop_ID", how="right")
line_2021.drop(columns=['Data_SEQ_x', 'Data_SEQ_y', 'Month_x', 'Month_y', "RT_Stop_Ord","Stop_YN","Turn_CA"], inplace=True)
line_2021 = pd.merge(line_2021, Route_bus_2021, left_on="RT_ID", right_on="RT_ID", how="left")
line_2021.drop(columns=['Month','MTC_Region', 'Data_SEQ', "RT_Desc", "RT_Group", "Region_CD"], inplace=True)
line_2021 = line_2021[line_2021['Stop_NM'].isin(s_list)]
line_21 = pd.DataFrame(pd.pivot_table(line_2021, index=['Stop_ID', 'Stop_NM', "RT_ID", "RT_NM"], values='Start', aggfunc=len)).reset_index()
line_21.drop(columns=['Start'], inplace=True)
line_21

,Stop_ID,Stop_NM,RT_ID,RT_NM
0,BS_1100_100000001,종로2가사거리,BR_1100_100100073,470
1,BS_1100_100000001,종로2가사거리,BR_1100_100100585,N37
2,BS_1100_100000001,종로2가사거리,BR_1100_123000010,741
3,BS_1100_100000002,창경궁.서울대학교병원,BR_1100_100100007,102
4,BS_1100_100000002,창경궁.서울대학교병원,BR_1100_100100009,104
...,...,...,...,...
251428,BS_TAGO_YSB8360,서창중학교,BR_TAGO_YSB5017008002,법서17-서창중경유
251429,BS_TAGO_YSB8360,서창중학교,BR_TAGO_YSB5025008000,웅진25
251430,BS_TAGO_YSB8360,서창중학교,BR_TAGO_YSB5025008001,웅진25-서창고후문경유
251431,BS_TAGO_YSB8360,서창중학교,BR_TAGO_YSB5025008002,웅진25-북부마을행


In [51]:
line_2020 = pd.merge(Stop_bus_2020, RouteStop_bus_2020[RouteStop_bus_2020["Stop_YN"] == "Y"], on="Stop_ID", how="right")
line_2020.drop(columns=['Data_SEQ_x', 'Data_SEQ_y', 'Month_x', 'Month_y', "RT_Stop_Ord","Stop_YN","Turn_CA"], inplace=True)
line_2020 = pd.merge(line_2020, Route_bus_2020, left_on="RT_ID", right_on="RT_ID", how="left")
line_2020.drop(columns=['Month','MTC_Region', 'Data_SEQ', "RT_Desc", "RT_Group", "Region_CD","Region_NM"], inplace=True)
line_2020 = line_2020[line_2020['Stop_NM'].isin(s_list)]
line_20 = pd.DataFrame(pd.pivot_table(line_2020, index=['Stop_ID', 'Stop_NM', "RT_ID", "RT_NM"], values='Start', aggfunc=len)).reset_index()
line_20.drop(columns=['Start'], inplace=True)
line_20

,Stop_ID,Stop_NM,RT_ID,RT_NM
0,BS_1100_100000001,종로2가사거리,BR_1100_100100073,470
1,BS_1100_100000001,종로2가사거리,BR_1100_100100585,N37
2,BS_1100_100000001,종로2가사거리,BR_1100_123000010,741
3,BS_1100_100000002,창경궁.서울대학교병원,BR_1100_100100007,102
4,BS_1100_100000002,창경궁.서울대학교병원,BR_1100_100100009,104
...,...,...,...,...
248433,BS_TAGO_YSB8337,경동아파트,BR_TAGO_YSB5007008003,웅진7
248434,BS_TAGO_YSB8337,경동아파트,BR_TAGO_YSB5007008004,웅진7
248435,BS_TAGO_YSB8360,서창중학교,BR_TAGO_YSB5017008002,법서17
248436,BS_TAGO_YSB8360,서창중학교,BR_TAGO_YSB5020008007,웅진20


In [52]:
line_2019 = pd.merge(Stop_bus_2019, RouteStop_bus_2019[RouteStop_bus_2019["Stop_YN"] == "Y"], on="Stop_ID", how="right")
line_2019.drop(columns=['Data_SEQ_x', 'Data_SEQ_y', 'Month_x', 'Month_y', "RT_Stop_Ord","Stop_YN","Turn_CA"], inplace=True)
line_2019 = pd.merge(line_2019, Route_bus_2019, left_on="RT_ID", right_on="RT_ID", how="left")
line_2019.drop(columns=['Month','MTC_Region', 'Data_SEQ', "RT_Desc", "RT_Group", "Region_CD","Region_NM"], inplace=True)
line_2019 = line_2019[line_2019['Stop_NM'].isin(s_list)]
line_19 = pd.DataFrame(pd.pivot_table(line_2019, index=['Stop_ID', 'Stop_NM', "RT_ID", "RT_NM"], values='Start', aggfunc=len)).reset_index()
line_19.drop(columns=['Start'], inplace=True)
line_19

,Stop_ID,Stop_NM,RT_ID,RT_NM
0,BS_1100_100000001,종로2가사거리,BR_1100_100100073,470
1,BS_1100_100000001,종로2가사거리,BR_1100_100100585,N37
2,BS_1100_100000001,종로2가사거리,BR_1100_123000010,741
3,BS_1100_100000002,창경궁.서울대학교병원,BR_1100_100100007,102
4,BS_1100_100000002,창경궁.서울대학교병원,BR_1100_100100009,104
...,...,...,...,...
248231,BS_TAGO_YSB8337,경동아파트,BR_TAGO_YSB5021008000,웅진21
248232,BS_TAGO_YSB8337,경동아파트,BR_TAGO_YSB5021008001,웅진21-웅상중고경유
248233,BS_TAGO_YSB8360,서창중학교,BR_TAGO_YSB5017008002,법서17-서창중경유
248234,BS_TAGO_YSB8360,서창중학교,BR_TAGO_YSB5020008007,웅진20-서창중행


# 6. 노선별 운행횟수
- 1) Stop_rail과 StopTime 데이터를 "Stop_ID" 를 기준으로 병합
- 2) 필요없는 컬럼 제거
- 3) 피벗테이블을 사용해 노선별 운행 회수 계산

In [55]:
me_2021 = pd.merge(Stop_bus_2021, StopTime_bus_2021, left_on="Stop_ID", right_on="Stop_ID", how="right")
bus_stop_count_2021 = pd.DataFrame(pd.pivot_table(data=me_2021, index=['Stop_ID', 'Stop_NM', "RT_ID"],values="ARS_ID",aggfunc=len)).reset_index()
bus_stop_count_2021.rename(columns = {'ARS_ID':'운행횟수'}, inplace = True)
bus_stop_count_2021

,Stop_ID,Stop_NM,RT_ID,운행횟수
0,BS_1100_100000001,종로2가사거리,BR_1100_100100073,137
1,BS_1100_100000001,종로2가사거리,BR_1100_100100585,8
2,BS_1100_100000001,종로2가사거리,BR_1100_123000010,132
3,BS_1100_100000002,창경궁.서울대학교병원,BR_1100_100100007,134
4,BS_1100_100000002,창경궁.서울대학교병원,BR_1100_100100009,158
...,...,...,...,...
493878,BS_TAGO_YSB8414,덕계사거리,BR_TAGO_YSB8003007002,2
493879,BS_TAGO_YSB8417,회야초등학교,BR_TAGO_YSB5007008000,52
493880,BS_TAGO_YSB8417,회야초등학교,BR_TAGO_YSB5007008002,4
493881,BS_TAGO_YSB8418,회야초등학교,BR_TAGO_YSB5007008000,52


In [58]:
me_2020 = pd.merge(Stop_bus_2020, StopTime_bus_2020, left_on="Stop_ID", right_on="Stop_ID", how="right")
bus_stop_count_2020 = pd.DataFrame(pd.pivot_table(data=me_2020, index=['Stop_ID', 'Stop_NM', "RT_ID"],values="ARS_ID",aggfunc=len)).reset_index()
bus_stop_count_2020.rename(columns = {'ARS_ID':'운행횟수'}, inplace = True)
bus_stop_count_2020

,Stop_ID,Stop_NM,RT_ID,운행횟수
0,BS_1100_100000001,종로2가사거리,BR_1100_100100073,137
1,BS_1100_100000001,종로2가사거리,BR_1100_100100585,8
2,BS_1100_100000001,종로2가사거리,BR_1100_123000010,132
3,BS_1100_100000002,창경궁.서울대학교병원,BR_1100_100100007,134
4,BS_1100_100000002,창경궁.서울대학교병원,BR_1100_100100009,158
...,...,...,...,...
490767,BS_TAGO_YSB8414,덕계사거리,BR_TAGO_YSB5007008000,33
490768,BS_TAGO_YSB8414,덕계사거리,BR_TAGO_YSB5007008001,1
490769,BS_TAGO_YSB8414,덕계사거리,BR_TAGO_YSB5007008002,2
490770,BS_TAGO_YSB8414,덕계사거리,BR_TAGO_YSB5007008003,1


In [59]:
me_2019 = pd.merge(Stop_bus_2019, StopTime_bus_2019, left_on="Stop_ID", right_on="Stop_ID", how="right")
bus_stop_count_2019 = pd.DataFrame(pd.pivot_table(data=me_2019, index=['Stop_ID', 'Stop_NM', "RT_ID"],values="ARS_ID",aggfunc=len)).reset_index()
bus_stop_count_2019.rename(columns = {'ARS_ID':'운행횟수'}, inplace = True)
bus_stop_count_2019

,Stop_ID,Stop_NM,RT_ID,운행횟수
0,BS_1100_100000001,종로2가사거리,BR_1100_100100073,134
1,BS_1100_100000001,종로2가사거리,BR_1100_100100585,8
2,BS_1100_100000001,종로2가사거리,BR_1100_123000010,132
3,BS_1100_100000002,창경궁.서울대학교병원,BR_1100_100100007,134
4,BS_1100_100000002,창경궁.서울대학교병원,BR_1100_100100009,176
...,...,...,...,...
471911,BS_TAGO_YSB8401,서창산업단지입구,BR_TAGO_YSB5014008002,1
471912,BS_TAGO_YSB8401,서창산업단지입구,BR_TAGO_YSB5014008003,8
471913,BS_TAGO_YSB8401,서창산업단지입구,BR_TAGO_YSB5014008004,3
471914,BS_TAGO_YSB8401,서창산업단지입구,BR_TAGO_YSB5018008000,1


---
# 7. 데이터 합치기(연도별로)
- 위에서 만든 모든 파일을 지하철 역명을 기준으으로 합쳐준다.

In [61]:
stop_time = pd.merge(bus_stop_count_2021, line_21, on=['Stop_ID', 'Stop_NM', "RT_ID"], how='outer')
bus_21 = pd.merge(stop_time, bus_stop, on="Stop_NM", how="left")
bus_21.to_csv('./aa/21년버스.csv', encoding='utf-8')

In [62]:
stop_time20 = pd.merge(bus_stop_count_2020, line_20, on=['Stop_ID', 'Stop_NM', "RT_ID"], how='outer')
bus_20 = pd.merge(stop_time20, bus_stop, on="Stop_NM", how="left")
bus_20.to_csv('./aa/20년버스.csv', encoding='utf-8')

In [63]:
stop_time19 = pd.merge(bus_stop_count_2019, line_19, on=['Stop_ID', 'Stop_NM', "RT_ID"], how='outer')
bus_19 = pd.merge(stop_time19, bus_stop, on="Stop_NM", how="left")
bus_19.to_csv('./aa/19년버스.csv', encoding='utf-8')